<a href="https://colab.research.google.com/github/georgef3/Visuals/blob/main/article_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Article Scraper

Article scraper using the newspaper3k library.

In [ ]:
!pip3 install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 74.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13541 sha256=f30b804521eb0d8eb3ea98abc8865888e4747e33167fbefd0ee092211d9c9c11
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=1907d22261e8de782472860b55d7f89751d741cd6a4296d31743

In [ ]:
import pandas as pd
# import newspaper3k
from newspaper import Article
from bs4 import BeautifulSoup

In [ ]:
# List of articles to scrape
# TODO: Try empty list/url, broken links, different languages, etc.
article_urls = ["http://fox13now.com/2013/12/30/new-year-new-laws-obamacare-pot-guns-and-drones/",
                "https://www.rt.com/russia/520785-india-approves-sputnikv-vaccine/",
                "https://www.cnn.com/2021/04/12/europe/ukraine-zelensky-front-lines/index.html",
                "https://actualidad.rt.com/actualidad/389125-fotos-satelite-portaviones-china-meridional"
                ]
print(f"There are {len(article_urls)} articles to scrape.")

# Dictionary to save scraper content into
articles_dict = {"url": [],
                 "title": [],
                 "authors": [],
                 "publish_date": [],
                 "text": [],
                 "top_image": [],
                 "movies": [],
                 "tags": [],
                 "description": [],
                 "keywords": [],
                 "summary": [],
                 "html": [],
                 "article_html": [],
                 "meta_lang": [],
                 "facebook": [],
                 "twitter": [],
                 "scraped_urls": []}
# TODO: Get language

# list of urls that did not get scraped
not_scraped = []

There are 4 articles to scrape.


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Iterate through each article url and scrape content
for url in article_urls:

    # Get article
    article = Article(url, keep_article_html=True)

    # Check if valid URL
    if not article.is_valid_url():
        print('{0} is not a valid url.'.format(url))
        not_scraped.append(url)

    else:

        # Download article
        article.download()
        #print(article.html)   # The article content is returned as an HTML

        # Parse HTML that was download
        article.parse()

        # Get keywords and summary of article
        article.nlp()

        # Add article data into dictionary
        articles_dict["url"].append(url)
        articles_dict["title"].append(article.title)
        articles_dict["authors"].append(article.authors)
        articles_dict["publish_date"].append(article.publish_date)
        articles_dict["text"].append(article.text)
        articles_dict["top_image"].append(article.top_image)
        articles_dict["movies"].append(article.movies)
        articles_dict["tags"].append(article.tags)
        articles_dict["keywords"].append(article.keywords)
        articles_dict["summary"].append(article.summary)
        articles_dict['html'].append(article.article_html)
        articles_dict['article_html'].append(article.article_html)
        articles_dict['description'].append(article.meta_description)
        articles_dict['meta_lang'].append(article.meta_lang)

        # Extract components of the metadata - might just be worth adding this as its own column
        md = article.meta_data
        try:    # Facebook page id
            articles_dict['facebook'].append(md['fb']['pages'])
        except:
            articles_dict['facebook'].append(None)

        try:    # Twitter page id
            articles_dict['twitter'].append(md['twitter']['site'])
        except:
            articles_dict['twitter'].append(None)

        # Extract urls from html
        soup = BeautifulSoup(article.html, 'html.parser')

        scraped_urls = []
        for link in soup.find_all('a'):
            scraped_urls.append(link.get('href'))

        articles_dict["scraped_urls"].append(scraped_urls)

# Convert articles dictionary into a pandas dataframe
articles_df = pd.DataFrame.from_dict(articles_dict)

In [ ]:
print('Scraped {0} articles. {1} urls were not scraped.'.format(len(articles_df), len(not_scraped)))
articles_df

Scraped 4 articles. 1 urls were not scraped.


,url,title,authors,publish_date,text,top_image,movies,tags,description,keywords,summary,html,article_html,meta_lang,facebook,twitter,scraped_urls
0,http://fox13now.com/2013/12/30/new-year-new-la...,"New Year, new laws: Obamacare, pot, guns and d...",[],2013-12-30,By Leigh Ann Caldwell\n\nWASHINGTON (CNN) — No...,http://fox13now.com/apple-touch-icon.png,[],{},By Leigh Ann Caldwell CNN WASHINGTON (CNN) — N...,"[latest, guns, wage, laws, drones, minimum, st...",Oregon: Family leave in Oregon has been expand...,<div><p>By Leigh Ann Caldwell</p><p>WASHINGTON...,<div><p>By Leigh Ann Caldwell</p><p>WASHINGTON...,en,75831650585,@fox13,"[/, /weather/alerts, /weather/school-closings-..."
1,https://www.rt.com/russia/520785-india-approve...,Sputnik V approved in India as New Delhi opens...,[Russia Today],NaT,Reports say that India has given the go-ahead ...,https://cdni.rt.com/files/2021.04/article/6074...,[],"{Russia news, Sputnik V news}",Reports say that India has given the go-ahead ...,"[v, opens, reddys, covid19, delhi, russian, ef...",Reports say that India has given the go-ahead ...,"<div><p class=""article__summary summary "">\n\n...","<div><p class=""article__summary summary "">\n\n...",en,326683984410,@RT_com,"[//arabic.rt.com/, //actualidad.rt.com/, //rus..."
2,https://www.cnn.com/2021/04/12/europe/ukraine-...,Ukraine's President heads to the trenches as R...,[Exclusive Matthew Chance],2021-04-12,On the front lines in eastern Ukraine (CNN) An...,https://cdn.cnn.com/cnnnext/dam/assets/2104111...,[],{},Tens of thousands of Russian troops gathered a...,"[ukrainian, heads, lines, russia, ukraines, tr...","""If I visit a military base, the guys at the v...",<div><p>On the front lines in eastern Ukraine ...,<div><p>On the front lines in eastern Ukraine ...,,"5550296508,18793419640",None,"[/, /world, /africa, /americas, /asia, /austra..."
3,https://actualidad.rt.com/actualidad/389125-fo...,Fotos por satélite muestran grupos de portavio...,[],NaT,Fotos por satélite muestran grupos de portavio...,https://cdni.rt.com/actualidad/public_images/2...,[],"{internacional, Estados Unidos, China, Armada}","A finales de la semana pasada, un grupo de buq...","[portaviones, la, uss, por, satélite, china, e...",Fotos por satélite muestran grupos de portavio...,"<div><p class=""ArticleView-title""><h1 class=""H...","<div><p class=""ArticleView-title""><h1 class=""H...",es,"296334033272,1617180398564925,137254146931398,...",@ActualidadRT,"[/, /en_vivo, /todas_las_noticias, /viral, /pr..."


In [ ]:
print(articles_df.iloc[0])

title           New Year, new laws: Obamacare, pot, guns and d...
authors                                                        []
publish_date                                  2013-12-30 00:00:00
text            By Leigh Ann Caldwell\n\nWASHINGTON (CNN) — No...
top_image                http://fox13now.com/apple-touch-icon.png
movies                                                         []
keywords        [obamacare, laws, latest, family, state, pot, ...
summary         Oregon: Family leave in Oregon has been expand...
Name: 0, dtype: object


In [ ]:
scraped_urls = articles_df.explode('scraped_urls')
scraped_urls.head()

,url,title,authors,publish_date,text,top_image,movies,tags,description,keywords,summary,html,article_html,meta_lang,facebook,twitter,scraped_urls
0,http://fox13now.com/2013/12/30/new-year-new-la...,"New Year, new laws: Obamacare, pot, guns and d...",[],2013-12-30,By Leigh Ann Caldwell\n\nWASHINGTON (CNN) — No...,http://fox13now.com/apple-touch-icon.png,[],{},By Leigh Ann Caldwell CNN WASHINGTON (CNN) — N...,"[laws, leave, obamacare, pot, wage, family, mi...",Oregon: Family leave in Oregon has been expand...,<div><p>By Leigh Ann Caldwell</p><p>WASHINGTON...,<div><p>By Leigh Ann Caldwell</p><p>WASHINGTON...,en,75831650585,@fox13,/
0,http://fox13now.com/2013/12/30/new-year-new-la...,"New Year, new laws: Obamacare, pot, guns and d...",[],2013-12-30,By Leigh Ann Caldwell\n\nWASHINGTON (CNN) — No...,http://fox13now.com/apple-touch-icon.png,[],{},By Leigh Ann Caldwell CNN WASHINGTON (CNN) — N...,"[laws, leave, obamacare, pot, wage, family, mi...",Oregon: Family leave in Oregon has been expand...,<div><p>By Leigh Ann Caldwell</p><p>WASHINGTON...,<div><p>By Leigh Ann Caldwell</p><p>WASHINGTON...,en,75831650585,@fox13,/weather/alerts
0,http://fox13now.com/2013/12/30/new-year-new-la...,"New Year, new laws: Obamacare, pot, guns and d...",[],2013-12-30,By Leigh Ann Caldwell\n\nWASHINGTON (CNN) — No...,http://fox13now.com/apple-touch-icon.png,[],{},By Leigh Ann Caldwell CNN WASHINGTON (CNN) — N...,"[laws, leave, obamacare, pot, wage, family, mi...",Oregon: Family leave in Oregon has been expand...,<div><p>By Leigh Ann Caldwell</p><p>WASHINGTON...,<div><p>By Leigh Ann Caldwell</p><p>WASHINGTON...,en,75831650585,@fox13,/weather/school-closings-delays
0,http://fox13now.com/2013/12/30/new-year-new-la...,"New Year, new laws: Obamacare, pot, guns and d...",[],2013-12-30,By Leigh Ann Caldwell\n\nWASHINGTON (CNN) — No...,http://fox13now.com/apple-touch-icon.png,[],{},By Leigh Ann Caldwell CNN WASHINGTON (CNN) — N...,"[laws, leave, obamacare, pot, wage, family, mi...",Oregon: Family leave in Oregon has been expand...,<div><p>By Leigh Ann Caldwell</p><p>WASHINGTON...,<div><p>By Leigh Ann Caldwell</p><p>WASHINGTON...,en,75831650585,@fox13,/live
0,http://fox13now.com/2013/12/30/new-year-new-la...,"New Year, new laws: Obamacare, pot, guns and d...",[],2013-12-30,By Leigh Ann Caldwell\n\nWASHINGTON (CNN) — No...,http://fox13now.com/apple-touch-icon.png,[],{},By Leigh Ann Caldwell CNN WASHINGTON (CNN) — N...,"[laws, leave, obamacare, pot, wage, family, mi...",Oregon: Family leave in Oregon has been expand...,<div><p>By Leigh Ann Caldwell</p><p>WASHINGTON...,<div><p>By Leigh Ann Caldwell</p><p>WASHINGTON...,en,75831650585,@fox13,/live
